# First ML Trials _(without structural info)_

* create a spark session & load data

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('BApredsV1').getOrCreate()

# load the data
data = spark.read.csv("bioavailability_data_final.csv",inferSchema=True,sep=',',header=True)

* <font color='gray'> Regarding data labels, recall that: </font>
  * <font color='gray'> The data has 5 labels: </font>
      *  <font color='gray'> **1 continuous label:** the BA percentage (_the original label_) </font> 
      *  <font color='gray'>**4 categorical labels:** the discretized groups of BA percentage </font> <br>
  * <font color='gray'> The categorical label columns were created by dividing _BA percentage_ into groups. The _"bucketing"/discretization ranges_ for these divisions were determined based on the review of histogram distributions and label statistics (_e.g.,_ grouped label mean & stdev) </font>
<br>
* for comparison with the manually-labeled categorical columns, an additional categorical label column will be added using Spark's built-in **QuantileDiscretizer**

In [ ]:
from pyspark.ml.feature import QuantileDiscretizer
import pandas as pd
qd5 = QuantileDiscretizer(numBuckets=5,inputCol='BA_pct',outputCol='label_QD5')
data = qd5.fit(data).transform(data)

* <font color='blue'> _For a quick review of all data labels, let's display them in a grouped DF:_ </font>

In [ ]:
''' # FINAL LABELS:
# ...1 continuous label column
# ...5 categorical label columns
# ---- label1 has 3 categories          -->                     low   /  mid   / high 
# ---- label2 has 4 categories          -->          very low / low       /      high / very high
# ---- the rest have 5 categories       -->          very low / low / moderate / high / very high
'''
from pyspark.sql import functions as F

data_wLabels.select(['BA_pct','label_QD5','label1','label2','label3a','label3b'])\
                .groupby('label_QD5','label3a','label3b','label2','label1')\
                    .agg(F.count('BA_pct'),F.min('BA_pct'),F.max('BA_pct'))\
                        .orderBy('max(BA_pct)')\
                            .show()

#### first regression and classification test

* prepare feature vectors

In [ ]:
cols_to_feature = ['MolWt','ExactMolWt','qed','MolLogP','MolMR','VSA_total','LabuteASA','TPSA',
                   'MaxPartialCharge','MinPartialCharge','MaxAbsPartialCharge','MinAbsPartialCharge',
                   'NumHAcceptors','NumHDonors','HeavyAtomCount','NumHeteroatoms','NumRotatableBonds',
                   'NHOHCount','NOCount','FractionCSP3','RingCount','NumAliphaticRings','NumAromaticRings',
                   'NumAliphaticHeterocycles','NumAromaticHeterocycles','NumSaturatedHeterocycles',
                   'NumSaturatedRings','BalabanJ','BertzCT','HallKierAlpha',
                   'fracVSA_PEOE01','fracVSA_PEOE02','fracVSA_PEOE03','fracVSA_PEOE04','fracVSA_PEOE05',
                   'fracVSA_PEOE06','fracVSA_PEOE07','fracVSA_PEOE08','fracVSA_PEOE09','fracVSA_PEOE10',
                   'fracVSA_PEOE11','fracVSA_PEOE12','fracVSA_PEOE13','fracVSA_PEOE14',
                   'fracVSA_SMR01','fracVSA_SMR02','fracVSA_SMR03','fracVSA_SMR04','fracVSA_SMR05',
                   'fracVSA_SMR06','fracVSA_SMR07','fracVSA_SMR08','fracVSA_SMR09','fracVSA_SMR10',
                   'fracVSA_SlogP01','fracVSA_SlogP02','fracVSA_SlogP03','fracVSA_SlogP04',
                   'fracVSA_SlogP05','fracVSA_SlogP06','fracVSA_SlogP07','fracVSA_SlogP08',
                   'fracVSA_SlogP09','fracVSA_SlogP10','fracVSA_SlogP11','fracVSA_SlogP12']

from pyspark.ml.linalg import Vector
from pyspark.ml.feature import (VectorAssembler,VectorIndexer)

vec_assembler = VectorAssembler(inputCols = cols_to_feature, outputCol='features')
data_w_features = vec_assembler.transform(data)

* index/encode categorical columns

In [ ]:
from pyspark.ml.feature import (StringIndexer,OneHotEncoder)

label_quant0 = 'BA_pct'

label_cat0_vector = OneHotEncoder(inputCol='label_QD5',outputCol='label_cat0_vector')

label_cat1_index = StringIndexer(inputCol='label1',outputCol='label_cat1_index')
label_cat1_vector = OneHotEncoder(inputCol='label_cat1_index',outputCol='label_cat1_vector')

label_cat2_index = StringIndexer(inputCol='label2',outputCol='label_cat2_index')
label_cat2_vector = OneHotEncoder(inputCol='label_cat2_index',outputCol='label_cat2_vector')

label_cat3_index = StringIndexer(inputCol='label3a',outputCol='label_cat3_index')
label_cat3_vector = OneHotEncoder(inputCol='label_cat3_index',outputCol='label_cat3_vector')

label_cat4_index = StringIndexer(inputCol='label3b',outputCol='label_cat4_index')
label_cat4_vector = OneHotEncoder(inputCol='label_cat4_index',outputCol='label_cat4_vector')

from pyspark.ml import Pipeline

data_pipeline = Pipeline(stages=[label_cat0_vector,
                                 label_cat1_index,label_cat1_vector,
                                 label_cat2_index,label_cat2_vector,
                                 label_cat3_index,label_cat3_vector,
                                 label_cat4_index,label_cat4_vector])

data_w_features = data_w_features.select(['Name','BA_pct','label_QD5','label1','label2','label3a','label3b','features'])
data_prefinal = data_pipeline.fit(data_w_features).transform(data_w_features)


In [ ]:
# clean up data
data_prefinal2 = data_prefinal.withColumnRenamed('BA_pct','label_q0')
data_prefinal2 = data_prefinal2.withColumnRenamed('label_QD5','label_cat0')
data_prefinal2 = data_prefinal2.withColumnRenamed('label_cat1_index','label_cat1')
data_prefinal2 = data_prefinal2.withColumnRenamed('label_cat2_index','label_cat2')
data_prefinal2 = data_prefinal2.withColumnRenamed('label_cat3_index','label_cat3')
data_prefinal2 = data_prefinal2.withColumnRenamed('label_cat4_index','label_cat4')

data_final = data_prefinal2.select(['Name',
                                    'label_q0',
                                    'label_cat0','label_cat1',
                                    'label_cat2','label_cat3','label_cat4',
                                    'features'])

* test a linear regression model

In [ ]:
subset_q0 = data_final.select(['label_q0','features'])
train1_q0,test1_q0 = subset_q0.randomSplit([0.7,0.3])

from pyspark.ml.regression import LinearRegression
lm_A = LinearRegression(featuresCol='features',labelCol='label_q0',predictionCol='prediction')

lmModel_1A = lm_A.fit(train1_q0)
lmResults1A = lmModel_1A.evaluate(test1_q0)

print(lmResults1A.rootMeanSquaredError, lmResults1A.r2)

<font color='purple'> ***Observations:*** </font> <br>
...
<br>
<font color='orange'> ***Next step:*** </font> <br>
...

* test a logistic regression model using `label_cat0` (Spark's QuantileDiscretizer label column)

In [ ]:
def log_reg_test(dataset,modelname,labelName,featuresName,eval_comparison):
    ''' # Select and split data 
    '''
    subset = dataset.select([labelName,featuresName])
    train,test = subset.randomSplit([0.7,0.3])

    ''' # Instantiate and run model 
    '''
    from pyspark.ml.classification import LogisticRegression
    lr = LogisticRegression(featuresCol=featuresName,labelCol=labelName,predictionCol='prediction')

    mymodel = lr.fit(train)
    myresults = mymodel.transform(test)

    ''' # Evaluate results on multiple metrics, output to df
    '''
    datasetName = myresults

    from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
    multiEvaluator = MulticlassClassificationEvaluator(labelCol=labelName, predictionCol="prediction")
    binEvaluator = BinaryClassificationEvaluator(labelCol=labelName, rawPredictionCol="prediction")

    evalMetrics = {binEvaluator:['areaUnderROC','areaUnderPR'], 
                   multiEvaluator:['f1','weightedPrecision','weightedRecall','accuracy']}
    evaluation = []
    for each_evaluator in [binEvaluator,multiEvaluator]:
        evaluator = each_evaluator
        for each_metric in evalMetrics[evaluator]:        
            metric = each_metric
            result = evaluator.evaluate(datasetName, {evaluator.metricName: metric})
            evaluation.append((metric,result))

    column0 = [x for x,y in evaluation]
    column1 = [y for x,y in evaluation]
    eval_comparison['metric'] = column0
    eval_comparison[modelname] = column1

    return eval_comparison

In [ ]:
eval_comparison = pd.DataFrame()
dataset = data_final
featuresName = 'features'
labelName = 'label_cat0'
modelname = 'lr_cat0'

eval_comparison = log_reg_test(dataset,modelname,labelName,featuresName,eval_comparison)

eval_comparison.head(6)

<font color='purple'> ***Observations:*** </font> <br>
...
<br>
<font color='orange'> ***Next step:*** </font> <br>
...

* test a logistic regression model using `label_cat1` (3-category BA labels: _low, medium, high_)

In [ ]:
dataset = data_final
featuresName = 'features'
labelName = 'label_cat1'
modelname = 'lr_cat1'

eval_comparison = log_reg_test(dataset,modelname,labelName,featuresName,eval_comparison)

eval_comparison.head(6)

<font color='purple'> ***Observations:*** </font> <br>
...
<br>
<font color='orange'> ***Next step:*** </font> <br>
...